# Esther's Text Processing Notebook Using Spacy

This notebook will focus on using Spacy for natural language processing.

### Importing and Installing

In [12]:
# Run in terminal for importing english pipeline, use of GPU, to train models
'''
!pip install -U pip setuptools wheel
!pip install -U 'spacy[cuda11x,transformers,lookups]'
!python -m spacy download en_core_web_trf
'''

ERROR: Invalid requirement: "'spacy[cuda11x,transformers,lookups]'"
c:\Users\esthe\AppData\Local\Programs\Python\Python311\python.exe: No module named spacy


In [1]:
from tqdm import tqdm

In [ ]:

import pandas as pd
import time
import numpy as np
from multiprocessing import Process
import spacy
from spacy import displacy

In [8]:
# load the english model
import spacy
spacy.prefer_gpu()
nlp = spacy.load('en_core_web_trf')

C:\Users\esthe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
C:\Users\esthe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


### Retrieving the Dataset from Dropbox + Cleaning

In [63]:

import dropbox
import pandas as pd
import io
import os 
#from config import dropbox_access_token

dropbox_access_token = 'sl.B0OyrqIVh5-MnCgbx9Vp9ZBe3UVv496hSidwMqXTtkIW-733ex9vaUN8aIxTMWKXrY4DD_cIJVx9griph0OVJK3TCnKsqi64viEj09FrnBScUNMw464e7PLkadxwfrcdHfsIbksqfS9r'

# Initialize Dropbox client
dbx = dropbox.Dropbox(dropbox_access_token)

# Dropbox file path
dropbox_file_path = '/fake_real_ML_project_dataset.csv'

# Download the file
metadata, response = dbx.files_download(dropbox_file_path)

# Read the CSV file from the response content using io.BytesIO
df = pd.read_csv(io.BytesIO(response.content))

# Show the results
print("Dataframe loaded from Dropbox:")
print(df.head())

Dataframe loaded from Dropbox:
   Unnamed: 0                                            Content  \
0           0  new york reuters us environmental group sierra...   
1           1  washington reuters us air force asked industry...   
2           2  saturday paul ryan posted photo instagram phot...   
3           3  america keeps waiting word hillary indicted ob...   
4           4                   religion peace ht weasel zippers   

          Title  Type  
0  politicsNews  true  
1  politicsNews  true  
2          News  fake  
3      politics  fake  
4     left-news  fake  


In [ ]:
%run C:\Users\esthe\DSEI2100_Project\machine-learning-dse-i210-final-project-fake-news\data\external\ML_dataset_dropbox.py

In [64]:
df = df.dropna()

In [65]:
# remove extra column
df = df.drop('Unnamed: 0', axis = 1)

In [66]:
df.head()

,Content,Title,Type
0,new york reuters us environmental group sierra...,politicsNews,true
1,washington reuters us air force asked industry...,politicsNews,true
2,saturday paul ryan posted photo instagram phot...,News,fake
3,america keeps waiting word hillary indicted ob...,politics,fake
4,religion peace ht weasel zippers,left-news,fake


In [7]:
df.shape

(44266, 3)

## Using Spacy

This section of the notebook will focus on using Spacy to process and extract information relevant to our goal of creating a fake news classification model.

We will use GPU to do these tasks and time them in order to compare them to our NLTK base.

### Spacy Tokenizer

In [67]:
def tokenize_df(df):
    df['Tokens'] = df['Content'].apply(lambda x: [token.text for token in nlp(x)])
    return df

In [48]:
token_process = Process(target=tokenize_df, args=(df))

In [34]:
#token_process.terminate()

In [49]:
token_process.start()

In [50]:
token_process.join()

In [ ]:
print(df['Tokens'].head())

In [68]:

def tokenize_df(df):
    df['Tokens'] = df['Content'].apply(lambda x: [token.text for token in nlp(x)])
    return df

if __name__ == "__main__":

    # Define the process
    token_process = Process(target=tokenize_df, args=(df.copy(),))

    # Start the process
    token_process.start()

    # Wait for the process to finish
    token_process.join()

    # Print the DataFrame after the process completes
    print("Printing DataFrame:")
    print(df.head())


### Spacy Lemmatizer

In [11]:
# lemmatize the text (takes much longer to run than nltk)
def lemmatize_df(df):
    df['lem_toks_spcy'] = df['Tokens'].apply(lambda tokens: [token.lemma_ for token in tokens])

In [ ]:
lemmatize_process = Process()

In [ ]:
lemmatize_process.start()
lemmatize_process.join()
print(df.head())

### Spacy POS Tagging

In [ ]:
def pos_df(df):
    df['pos_tag'] = df['Tokens'].apply(lambda tokens: [token.pos_ for token in tokens])

### Spacy Parser

In [ ]:
doc = df['Tokens'].loc[1]

In [ ]:
displacy.render(doc, style='dep')

### Spacy Entity Recognizer

spaCy's statistical entity recognition system assigns labels to contigouos spans of tokens. It works to identify a variety of named numeric entities including companies, locations, organizations, and products

### Spacy Text Categorizer